

> Import libraries to use



In [ ]:
import numpy as np

>  # Introduction to numpy (Skip if you already are familiar)

>> Creating a 1D array

In [ ]:
a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


>> Creating a 2D array


In [ ]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


>> Creating an array full of zeros


In [ ]:
a = np.zeros(shape=(10))
print(a)
a = np.zeros(shape=(5,2))
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


>> Infinity in numpy

In [ ]:
print(np.inf)

inf


>> Max and Argmax

In [ ]:
a = np.array([2,1,4,3])
print(np.max(a))
print(np.argmax(a))

4
2


>> From list to Numpy

In [ ]:
l = [1,2,3,4]
print(l)
print(np.asarray(l))

[1, 2, 3, 4]
[1 2 3 4]


>> Random in numpy

In [ ]:
# Array of Random integers ranging from 1 to 10 (with any size you want)
a = np.random.randint(low=1, high=10, size=(5,2))
print(a)

# Array of random elements of a list with any size you want
a = np.random.choice([0,1,2], size=(2,))

[[6 4]
 [1 6]
 [7 5]
 [5 9]
 [4 2]]


>> Shapes in numpy

In [ ]:
a = np.random.randint(low=1, high=5, size=(4,2))
print(a.shape)
print(a)

# Reshape a to a vector of shape = (8,1)
a = a.reshape((8,1))
print(a.shape)
print(a)

(4, 2)
[[4 3]
 [3 2]
 [2 3]
 [2 2]]
(8, 1)
[[4]
 [3]
 [3]
 [2]
 [2]
 [3]
 [2]
 [2]]


# Pre-defined utilities

In [ ]:

int_to_char = {
    0 : 'u',
    1 : 'r',
    2 : 'd',
    3 : 'l'
}

policy_one_step_look_ahead = {
    0 : [-1,0],
    1 : [0,1],
    2 : [1,0],
    3 : [0,-1]
}

def policy_int_to_char(pi,n):

    pi_char = ['']

    for i in range(n):
        for j in range(n):

            if i == 0 and j == 0 or i == n-1 and j == n-1:

                continue

            pi_char.append(int_to_char[pi[i,j]])

    pi_char.append('')

    return np.asarray(pi_char).reshape(n,n)

# 1- Policy evaluation

In [6]:
"""
  This function should return the value function that follows the policy pi.
  Use the stopping criteria given in the problem statement.
"""
import numpy as np

def policy_evaluation(n, pi, v, gamma, threshold):
    while True:
        delta = 0
        for i in range(n):
            for j in range(n):
                if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                    continue
                v_old = v[i, j]
                action = pi[i, j]
                next_i, next_j = i + policy_one_step_look_ahead[action][0], j + policy_one_step_look_ahead[action][1]
                next_i = np.clip(next_i, 0, n - 1)
                next_j = np.clip(next_j, 0, n - 1)
                reward = -1
                v[i, j] = reward + gamma * v[next_i, next_j]
                delta = max(delta, abs(v[i, j] - v_old))
        if delta < threshold:
            break
    return v


# 2- Policy improvement

In [7]:
"""
  This function should return the new policy by acting in a greedy manner.
  The function should return as well a flag indicating if the output policy
  is the same as the input policy.

  Example:
    return new_pi, True if new_pi = pi for all states
    else return new_pi, False
"""
import numpy as np

def policy_improvement(n, pi, v, gamma):
    new_pi = np.zeros((n, n), dtype=int)
    pi_stable = True
    for i in range(n):
        for j in range(n):
            if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
                continue
            old_action = pi[i, j]
            action_values = []
            for action in range(4):
                next_i, next_j = i + policy_one_step_look_ahead[action][0], j + policy_one_step_look_ahead[action][1]
                next_i = np.clip(next_i, 0, n - 1)
                next_j = np.clip(next_j, 0, n - 1)
                reward = -1
                action_value = reward + gamma * v[next_i, next_j]
                action_values.append(action_value)
            best_action = np.argmax(action_values)
            new_pi[i, j] = best_action
            if old_action != best_action:
                pi_stable = False
    return new_pi, pi_stable


# 3- Policy Initialization

In [11]:
"""
  This function should return the initial random policy for all states.
"""
import numpy as np

def policy_initialization(n):
    pi = np.random.randint(0, 4, size=(n, n))
    return pi

# 4- Policy Iteration algorithm

In [12]:
import numpy as np

def policy_iteration(n, Gamma, threshold):
    pi = policy_initialization(n=n)
    v = np.zeros(shape=(n, n))

    while True:
        v = policy_evaluation(n=n, v=v, pi=pi, gamma=Gamma, threshold=threshold)
        new_pi, pi_stable = policy_improvement(n=n, pi=pi, v=v, gamma=Gamma)

        if pi_stable:
            break

        pi = new_pi

    return pi, v

# Main Code to Test

In [ ]:
n = 4
gamma_values = [0.8, 0.9, 1]
threshold = 1e-4

for gamma in gamma_values:
    optimal_policy, optimal_value_function = policy_iteration(n=n, Gamma=gamma, threshold=threshold)
    optimal_policy_char = policy_int_to_char(optimal_policy, n)

    print("Gamma =", gamma)
    print(optimal_policy_char)
    print(optimal_value_function)
    print()
